In [1]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
import nltk
import string
import pandas as pd
import re
#python -m spacy download en_core_web_sm

import os 

#nltk.download('punkt')
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle') 

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 5000)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_column',None)
pd.set_option('display.width', 5000)
pd.set_option('display.max_colwidth', -1)

In [3]:
# Set globals


# Need lower threshold that represents cheapest package. 
# Necessary to filter out noise from phone numbers, quantifiers, etc.
lower_threshold = 15

# Need upper threshold to help rule out discussion of wifi speeds. 
upper_threshold = 250

units = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen",
    ]

tens = ["", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]

scales = ["hundred", "thousand", "million", "billion", "trillion"]

no_package_moce_price = {'NO PRODUCTS': 4.99}

single_package_mode_price = {'VIDEO ONLY': 49.99, 'CDV ONLY': 29.99, 'XM ONLY': 30.00, 'HSD ONLY': 39.99
                            , 'XH ONLY' : 24.99
    
}

double_package_mode_price = {'HSD/XH': 39.99 , 'HSD/CDV': 59.99 , 'CDV/XH': 19.99, 'XH/XM': 24.99,
                            'VIDEO/CDV': 74.99 , 'CDV/XM': 25.00, 'VIDEO/XM': 54.99, 'HSD/XM': 39.99 , 
                            'VIDEO/XH': 29.99 , 'VIDEO/HSD': 49.99 
}

triple_package_mode_price = {'VIDEO/CDV/XH': 24.99 , 'VIDEO/HSD/XM': 89.99 , 'HSD/CDV/XM': 59.99 ,
                            'CDV/XH/XM': 24.10 , 'VIDEO/HSD/XH': 109.99 , 'HSD/XH/XM': 39.99 , 
                            'VIDEO/HSD/CDV': 119.99 , 'HSD/CDV/XH': 59.99 , 
                            'VIDEO/CDV/XM': 54.99, 'VIDEO/XH/XM': 24.99
                            
    
}

quad_package_mode_price = {'VIDEO/HSD/XH/XM': 109.99, 'VIDEO/HSD/CDV/XH': 24.99, 'HSD/CDV/XH/XM': 29.95, 
                          'VIDEO/CDV/XH/XM': 24.99, 'VIDEO/HSD/CDV/XM': 119.99
    
}

price_dict = dict(single_package_mode_price)
price_dict.update(double_package_mode_price) 
price_dict.update(triple_package_mode_price) 
price_dict.update(quad_package_mode_price) 

master_price_dict = {19.99: 'CDV/XH',
               24.10: 'CDV/XH/XM', 
               24.99: ('XH ONLY', 'XH/XM', 'VIDEO/CDV/XH', 'VIDEO/XH/XM',
                       'VIDEO/HSD/CDV/XH', 'VIDEO/CDV/XH/XM', 'CDV/XM'), 
                29.99: ('HSD/CDV/XH/XM', 'CDV ONLY', 'VIDEO/XH', 'XM ONLY'), 
               39.99: ('HSD ONLY', 'HSD/XH', 'HSD/XM', 'HSD/XH/XM'), 
               49.99: ('VIDEO ONLY', 'VIDEO/HSD'), 
               54.99: ('VIDEO/XM', 'VIDEO/CDV/XM'), 
               59.99: ('HSD/CDV', 'HSD/CDV/XM', 'HSD/CDV/XH'), 
               74.99: 'VIDEO/CDV', 
               89.99: 'VIDEO/HSD/XM', 
               109.99: ('VIDEO/HSD/XH', 'VIDEO/HSD/XH/XM'), 
               119.99: ('VIDEO/HSD/CDV', 'VIDEO/HSD/CDV/XM')
              }
               


# price_dict_rev = {v: k for k, v in price_dict.items()}
price_list = [19.99, 24.10, 24.99, 29.99, 39.99, 49.99, 54.99, 59.99, 74.99, 89.99, 109.99, 119.99]


text_transformation_dic={'home security system':'home-security-system',\
     'security system':'security-system',\
     'home security':'home-security',
     'set top box':'set-top-box',\
     'xfinity mobile':'xfinity-mobile',\
     'infinity mobile':'xfinity-mobile',\
     'land line':'landline',\
     'home phone':'home-phone',\
     'hot spot':'hotspot',\
     'home phones':'home-phone',\
     'quad play':'quad-play',\
     'double play':'double-play',\
     'triple play':'triple-play',\
     'single play':'single-play',\
     'performance pro':'performance-pro',\
     'performance plus':'performance-plus',\
     'performance starter':'performance-starter',\
     'amazon prime': 'amazon-prime',\
     'sling tv':'sling-tv',\
     'direct tv':'direct-tv',\
     'high definition':'high-definition'
    }


# Negative keywords to look for in text.
estimates = ['about', 'around', 'almost', 'nearly', 'between', 'approaching']
post_num = ['bytes', 'byte', 'megabytes', 'megabyte', 'gigabytes', 'gigabyte', 
            'terabytes', 'terabyte', 'channels', 'channel', 'charge', 'credit']

# Positive keywords to look for in text
key_num1 = 'ninety nine'
key_num2 = 'ninety five' 
key_words1 = ['package', 'promotion', 'offer', 'offering', 'bundle', 'bundles', 'month']
    
# Specific tier keywords. 
hsd=['speed','internet','megabyte','megabytes','megabits','modem','gigabytes','signal','wifi','network','router']
video=['tv','TV','channel','channels','set-top-box','dvr','hd','hbo','espn','box','boxes']
cdv=['landline','home-phone']
xm=['mobile','lines','hotspot','spot','xfinity-mobile']
xh=['security-system','home-security-system','home-security-system','home-security','security','camera','cameras']

# Other Categories of keywords
stream=['streaming','stream']
ott=['netflix','hulu','roku']
hsd_tier=['blast','performance','performance-pro','performance-plus','performance-starter','gigabit']
vid_tier=['latino','dvr','hd','high-definition']
packages=['quad-play','double-play','triple-play','single-play']

nlp = en_core_web_sm.load()



In [4]:
def replace_all(text, dic):
    for i, j in dic.items():
        src_str  = re.compile(i, re.IGNORECASE)
        text = src_str.sub(j,text)
    return text


In [5]:
# Converts text form of number to decimal.

def text2int(textnum, numwords={}):
    
    # Put everything in lower case, strip commas, and call missing_category_check on textnum. 
    textnum = textnum.lower()
    textnum = textnum.replace(',', '')
    textnum = missing_category_check(textnum)

    # Made True upon finding keywords from 'descriptors'
    take_ave = False 
    waiting_for_and = False
    
    # Stores values for calculating averages with 'to' and 'between' cases.
    keep = [] 

    # List of non number keywords to look for.
    descriptors = ['a', 'to', 'between','dollars', 'cents']
      
    if not numwords:      
    # And symbolizes addition
      numwords["and"] = (1, 0)
      for idx, word in enumerate(units):
          numwords[word] = (1, idx)
      for idx, word in enumerate(tens):
          numwords[word] = (1, idx * 10)
      for idx, word in enumerate(scales):  
          numwords[word] = (10 ** (idx * 3 or 2), 0)
        
    current = result = 0
    
    # For each word test if in 'descriptors or if a number word.'
    for word in textnum.split():
        if word not in numwords and word not in descriptors:
            continue 
        # 'a' in a number clause symbolizes '1'
        if word == 'a': 
            word = 'one' 
        # 'to' compares two values. When found will calculate average of two. 
        elif word == 'to': 
            take_ave = True 
            keep.append(current) 
            current = 0
            result = 0
            continue
        # 'between' compares two values in conjunction with 'and'. When found will calculate average of two. 
        elif word == 'between':
            waiting_for_and = True
            continue
        elif waiting_for_and and word == "and":
            waiting_for_and = False
            take_ave = True
            keep.append(current)
            current = 0
            result = 0
            continue
        # Anything after the word dollars needs to be treated as cents. 
        # Missing category_check will add cents to end on proper condition if speaker hasn't.  
        elif word == 'dollars':
            result = result + current 
            current = 0
            continue
        # Makes sure cents aren't being read as dollars. 
        elif word == 'cents':
            current = current * .01
            continue

        scale, increment = numwords[word]
        current = current * scale + increment
        if scale > 100:
            result += current
            current = 0
            
    # if 'to' or 'between' was found in clause, calculate average. Otherwise proceed normally.        
    if take_ave and keep[0] != 0: 
        return(result + current + sum(keep)) / (len(keep) + 1)
    else:
        return result + current



In [6]:
# Takes account for cases where speaker is speaking improperly including. 
# 1.) Not say hundred after give hundreds place 
# 2.) Saying something like thirty nine ninety nine rather than thiry nine dollars and ninety nine cents

def missing_category_check(textnum):
    change_made = False
    cent_added = False
    prev_word = ''
    count = 0
    words = textnum.split()
    for word in words:
        if prev_word in units and word in tens and count == 1:
            word = 'hundred ' + word
            change_made = True
            words[count] = word
        elif (prev_word in units or prev_word in tens) and word in tens and count > 1:
            word = 'dollars and ' + word
            cent_added = True
            change_made = True
            words[count] = word
        prev_word = word
        count += 1

    if change_made:
        s = " ".join(words)
        if cent_added:
            s = s + " cents"
        return s
    else:
        return textnum

        

In [7]:
# Determines how many sentences before and after the current one we can take in for context. 

def check_position(i, sentence_count):
    if i > 1:
        low = i - 2
    else: 
        low = 0
    if sentence_count - 1 - i > 1:
        high = i + 3 
    else: 
        high = sentence_count
    return([low, high])
    

In [8]:
# importance_analysis(context, X.text) 
# Shows on a scale of one to ten how important this price is. 

def price_importance_analysis(context, token): 
    # Removes all punctuation from context except for hyphens and token and splits into array.
    remove = string.punctuation.replace("-", "")
    context = context.lower().translate(str.maketrans('', '', remove))
    token = token.lower().translate(str.maketrans('', '', remove))
    
    # Default score is 0. 
    score = 0
    
    # Booleans to make sure each category only gets accounted for once. 
    estimate_found = False
    post_num_found = False
    key_words1_found = False 
    packages_found = False 
    
    # Look in context and token for above words. Token has more importance than context. 
    for word in context.split(): 
        if word in estimates and not estimate_found:
            if word in token: 
                score -= 2
            else: 
                score -= 1 
            estimate_found = True 
            continue
        if word in post_num and not post_num_found:
            if word in token: 
                score -= 2
            else: 
                score -= 1
            post_num_found = True
            continue 
        if word in key_words1 and not key_words1_found: 
            score += 2
            key_words1_found = True
            continue
        if word in packages and not packages_found:
            score += 2 
            packages_found = True
            continue

    # Ninety nine and ninety five are big indicators of package offerings. 
    if key_num1 in token or key_num2 in token:
        score += 4 
    elif key_num1 in context or key_num2 in context: 
        score += 2
    
    # Commas in token imply list of numbers. 
    if ',' in token: 
        score -= 3
    

    return score


# price_importance_analysis("IT'S A HUNDRED AND TWENTY DOLLARS A MONTH, UM, FOR THE TRIPLE-PLAY AND LET ME SEE ONE MORE THING", "a hundred and twenty")

In [9]:
# Forms guess based on price if score is above 2. 
def make_guess(price):
    
    # Finds closest value in price list to given price. 
    closest_num = min(price_list, key=lambda x:abs(x-price))
    
    # Returns packages that are most commonly priced here. 
    return master_price_dict[closest_num]
    

In [10]:
# Want only Cardinal and Money types. Gives context of two sentences before and after
# Don't want lists of numbers. Ignore tokens that are lists of number. 
# Capitalizes sentence that had mention in it. 

def extract_info(transcript_sentences):
    i = 0 
    guess = ''
    max_score = -10
    transcript_price = 0 
    transcript_score = 0 
    transcript_context = ''
    transcript_label = []
    transcript_score = 0 
    transcript_context = '' 
    sentence_count = len(transcript_sentences)
    while i < sentence_count:
        context_gauge = check_position(i, sentence_count) 
        lower_limit = context_gauge[0] 
        upper_limit = context_gauge[1]
        doc = nlp(transcript_sentences[i])
        
        for X in doc.ents: 
            # When comma inside X.text this implies list of numbers. Don't want these. 
            if X.label_ in ('CARDINAL', 'MONEY', 'QUANTITY') and X.text.count(',') < 1: 
                price = text2int(X.text)
                if price > lower_threshold and price < upper_threshold: 
                    score = price_importance_analysis(transcript_sentences[i], X.text)
                    context = [transcript_sentences[lower_limit:i], transcript_sentences[i].upper(), transcript_sentences[i+1:upper_limit]]
                    if score >= max_score: 
                        max_score = score 
                        transcript_price = price
                        transcript_score = score 
                        transcript_context = context
                        # transcript_context = transcript_sentences[i]
        i += 1
    if transcript_score > 3: 
        guess = make_guess(transcript_price) 
    else: 
        guess = 'NONE' 
    return [transcript_score, transcript_price, transcript_context, guess]
    
    

In [15]:
def main():
    rows_list = [] 
    error_count = 0 
    audio_directory = "/Users/mmanle240/Downloads/nlp_git/Audio_Transcription/Amazon/Transcripts2"
    for filename in os.listdir(audio_directory): 
        if filename.endswith(".txt"):
            print(filename)
            transcript_file_path = audio_directory + "/" + filename
            transcript_file = open(transcript_file_path, "r")
            transcript_contents = transcript_file.read().lower()
            transcript_contents = replace_all(transcript_contents, text_transformation_dic)
            doc = nlp(transcript_contents)
            transcript_sentences = tokenizer.tokenize(transcript_contents) 
            data = extract_info(transcript_sentences)  
            rows_list.append(data)
            transcript_file.close() 
    df = pd.DataFrame(rows_list)
    df.columns = ['Score', 'Price', 'Context', 'Guess']
    return df


In [16]:
df = main()

10041519431551386351.txt
23505598841550185660.txt
31700020911549746015.txt
10027320231549994007.txt
10041205621551108265.txt
10026463411550765825.txt
10037157521550610392.txt
10039564961551121005.txt
10034120831550938826.txt
10039480241550978032.txt
10024338221551288668.txt
10040048431551043596.txt
10036375781551394864.txt
10024490731550175745.txt
10034506321551379139.txt
10028177241550614885.txt
10033372161550852843.txt
10033146401549748506.txt
10035262901550951617.txt
10036392741550074374.txt
10029349891551298355.txt
10027198271550672004.txt
10033441881550101380.txt
10038056551549649810.txt
10032227751550248678.txt
10001377861551369299.txt
10037130651549737376.txt
10034232121550253442.txt
10026076441549661295.txt
10039153551549756670.txt
10026077961551118639.txt
10027015891551217984.txt
10033053781550506858.txt
10041082601550945002.txt
10022302631550499038.txt
10041580161550782597.txt
10026367471551391971.txt
10033200861551391735.txt
10024064341550948008.txt
10023088811549666899.txt


In [48]:
df

,Score,Price,Context,Guess
0,1,29.00,"[[she's looking., picks it.], SHE SAID I'LL GIVE YOU A HUNDRED MEGABYTES PLUS FOR THIRTY NINE BECAUSE I HAVE TWENTY NINE DOLLARS A MONTH NOW., [not okay., she said she would work in about twenty minutes.]]",NONE
1,0,28.00,"[[can i please have you verify the address of the location you're at?, uh, yes.], UH, TWENTY EIGHT., [oh., five west.]]",NONE
2,6,124.99,"[[okay, um, customer first, um, actually, we would like to pay the bills, but we told her you found out that the promotion provided the word i'm for him last december, twenty fourth of two thousand eighteen did not go through eventually on the account or did not, you know, updated in the system so i can see them., there is documentation, supposed.], DID WE WANNA ARCULLI COMING FROM YOUR DEPARTMENT FROM THEIR ATTENTION TEAM THAT, UM, THE THE CUSTOMER WAS GIVEN PROMOTIONAL PACKAGE OF ONE TWENTY FOUR NINETY NINE FOR PREFERRED PACKAGE AND THE CHANGES ACCORDING TO THE ACCOUNT THROUGH THEM A FORTY OR THIRTY APPROVAL., [what is we totally try to your cat customer steal, um, paying the regular rate on this tarp government., so i'm trying to get your assistance.]]","(VIDEO/HSD/CDV, VIDEO/HSD/CDV/XM)"
3,2,80.00,"[[you're working now., how how much is the offer again?], UM, I TRIED TO OFFER HIM THIRTY DOLLARS, BUT THEY ARE ABLE TO REVIEW THE HOUSE SIXTY TWENTY AS WELL., [but on the validation, um, it's really a valid charge, right?, yes.]]",NONE
4,4,100.00,"[[when you have the customer before submitting your order, i want to spend a few minutes confirming with you accurately capture the services you want., i'll ask you to review the order and then if it's correct, please approve it.], YOU WANT ORDERING THE X ONE FOR FOR TRIPLE TWENTY PACKAGE FOR AN INTRODUCTORY PRICE OF ONE HUNDRED TWENTY FOUR DOLLARS AND NINETY NINE CENTS PER MONTH FOR TWENTY FOUR MONTHS., [it includes digital preferred digital preferred stars, stream picks, digitalized outlets s v. c. did you laugh?, i hope you have a s v c. did you live out yet?]]","(VIDEO/HSD/XH, VIDEO/HSD/XH/XM)"
5,1,90.00,"[[looks like this is going to come out quite a little bit lower than probably what you were paying before, but i've got a couple of areas here that i need to clear up., so is it okay to call you back?], UM, MISS E A. I'M LOOKING AT IT BEING SOMEWHERE AROUND ABOUT EIGHTY OR NINETY DOLLARS A MONTH, YOU KNOW, IS WHAT I'M ESTIMATING IT TO BE CAN I CALL YOU BACK?, [is it the seven?, six five six four two six two nine zero yes, i'll be there alright give me about maybe five or ten minutes and i when you back let me clear this air out for me okay?]]",NONE
6,4,120.00,"[[okay, i want to make sure that there isn't any extra options., yeah, so that would be the best one.], IT'S A HUNDRED AND TWENTY DOLLARS A MONTH, UM, FOR THE TRIPLE-PLAY AND LET ME SEE ONE MORE THING., [what?, okay?]]","(VIDEO/HSD/CDV, VIDEO/HSD/CDV/XM)"
7,0,34.00,"[[and i don't get it, okay?, i was just trying to do read their policy.], SO FOR YOU TO REGISTER THAT YOU HAVE TO LOOK INTO YOUR JACKLEY BROWN EMAIL EXACTLY A DECADE BROWN THIRTY FOUR., [i'm here just to verify your email., yes.]]",NONE
8,4,24.99,"[[so if if you if it's just the three you doing, then he is., three years, two years at one.], TWENTY FOUR NINETY NINE AND ONE YEAR AT WORK., [forty nine., yes.]]","(XH ONLY, XH/XM, VIDEO/CDV/XH, VIDEO/XH/XM, VIDEO/HSD/CDV/XH, VIDEO/CDV/XH/XM, CDV/XM)"
9,0,0.00,,NONE


In [18]:
audio_directory = "/Users/mmanle240/Downloads/nlp_git/Audio_Transcription/Amazon/Transcripts2"
for filename in os.listdir(audio_directory): 
    if filename.endswith(".txt"):
        output = filename[:-4]
        print(output)

10041519431551386351
23505598841550185660
31700020911549746015
10027320231549994007
10041205621551108265
10026463411550765825
10037157521550610392
10039564961551121005
10034120831550938826
10039480241550978032
10024338221551288668
10040048431551043596
10036375781551394864
10024490731550175745
10034506321551379139
10028177241550614885
10033372161550852843
10033146401549748506
10035262901550951617
10036392741550074374
10029349891551298355
10027198271550672004
10033441881550101380
10038056551549649810
10032227751550248678
10001377861551369299
10037130651549737376
10034232121550253442
10026076441549661295
10039153551549756670
10026077961551118639
10027015891551217984
10033053781550506858
10041082601550945002
10022302631550499038
10041580161550782597
10026367471551391971
10033200861551391735
10024064341550948008
10023088811549666899
10024116651550778579
10023104301550533812
10029009591550081937
30700143981550014331
10028080171549663234
10027471241550164495
10033399911550869108
